## Questions

Are students asking conceptual or appllied questions?
Which topics are more prevalent? Where is the most confusion?
    - Topic Modeling
    - Run the same topic model over different intervals of time
What do in-person interactions not already tell us? 
What are areas that need attention in student understanding? 
    - Concrete evidence of topics where students are struggling
Probably don't look at response time
What are people afraid of asking about? (Anonymous posts from students)
Data:
    - Timestamp
    - Userid (student or instructor, throw out announcements)
    - Question or answer? 
    - Votes (good question)
    - Question Text
    
    
diff values of config?
Check status = active
transform to get length of upvote_ids

Question or Answer? 
Num votes/good question
where does anon equal yes

In [18]:
import glob
import pickle
import pandas as pd

In [19]:
dfs = glob.glob('data/dataframes/*.p')
print(dfs[0])

data/dataframes/lexokan@live.unc.edu_dataframe_j5wwaj87hvu6af.p


In [20]:
df = pd.DataFrame(pickle.load(open(posts[0], "rb")))
df

,id,created,type,folders,tags,is_announcement,history,children,tag_good,is_student,no_answer,num_children,num_favorites,num_revisions,unique_views,subject,text
0,jau1tihlbb82ou,2017-12-05 19:59:43+00:00,note,[general],"[general, instructor-note, pin]",1,"[{'anon': 'no', 'uid': 'hbpawagy1su7ok', 'subj...","[{'anon': 'stud', 'folders': [], 'data': {'emb...",[],False,NaN,1,1,1,244,Last Call For Style Regrade Requests,Any code style regrade requests should be in ...
1,jatlo9mj8eh16r,2017-12-05 12:27:44+00:00,note,[exams],"[exams, instructor-note, pin]",1,"[{'anon': 'no', 'uid': 'hbpawagy1su7ok', 'subj...",[],[],False,NaN,0,0,2,268,Final Exam Logistics,The final exam for our class will be at 8 AM ...
2,ja2v62y52r47iw,2017-11-16 19:23:45+00:00,note,[general],"[general, instructor-note, pin]",1,"[{'anon': 'no', 'uid': 'hbpawagy1su7ok', 'subj...",[],[],False,NaN,0,0,1,258,Material for recitation tomorrow,Below are links to the materials you&#39;ll n...
3,ja2josndfk6z7,2017-11-16 14:02:23+00:00,note,[general],"[general, instructor-note, pin]",1,"[{'anon': 'no', 'uid': 'hbpawagy1su7ok', 'subj...",[],[],False,NaN,0,2,1,257,Grading issue punchlist form,If you have any sort of grade / course issue ...
4,j9urk22tlgx5jy,2017-11-11 03:20:29+00:00,note,[assignment7],"[assignment7, instructor-note, pin]",1,"[{'anon': 'no', 'uid': 'hbpawagy1su7ok', 'subj...","[{'anon': 'stud', 'folders': [], 'data': {'emb...","[{'role': 'student', 'name': 'Justin Do', 'adm...",False,NaN,1,0,2,260,Autograder machine having problems,Something seems to be wrong with the machine....
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
850,j6mz3tjz9jy2cx,2017-08-22 02:30:32+00:00,note,[general],"[general, instructor-note]",0,"[{'anon': 'no', 'uid': 'hbpawagy1su7ok', 'subj...",[],[],False,NaN,0,11,5,302,Pre-Lecture Readings / Videos / Exercises,(8/24) Lecture 2 http://www.brighthub.c...
851,j6mvpt4f9yu3a4,2017-08-22 00:55:39+00:00,note,[general],"[general, instructor-note, past-class, private]",0,"[{'anon': 'no', 'uid': 'hbpawagy1su7ok', 'subj...",[],[],False,NaN,0,3,3,286,"Tutorials, Articles, Other On-line Resources",Basic tutorials on getting started with Java ...
852,j6mvofglgfc2v2,2017-08-22 00:54:35+00:00,note,[general],"[general, instructor-note, past-class, private]",0,"[{'anon': 'no', 'uid': 'hbpawagy1su7ok', 'subj...",[],[],False,NaN,0,3,3,281,Tools and Reference Documents,Java SDK http://www.oracle.com/technetwork...
853,j6f7s9qfj841y3,2017-08-16 16:11:20+00:00,note,"[assignment1, assignment2, assignment3, assign...","[assignment1, assignment2, assignment3, assign...",0,"[{'anon': 'no', 'uid': 'hbpawagy1su7ok', 'subj...",[],[],False,NaN,0,26,16,320,Assignment Schedule,All assignments are due at 11:59:59 PM ...
